In [ ]:
import os 
import re
import json
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.stats.multitest as smt

def sin_cos_24hr_encoder(df, col):
    """
    Given a dataframe and column 
    This function takes a dataframe and a column that contains hours of the day,
    and creates two new columns 'Hour_Sin' and 'Hour_Cos'. These columns contain
    the sine and cosine encodings of the time, which is a common technique used
    for cyclical features like time of day in machine learning models.

    The sine and cosine values are calculated such that they form a circle when plotted,
    which is useful for the model to recognize the cyclical nature of the data
    (e.g., 23 hours is close to 0 hours).

    Parameters:
    df (DataFrame): The pandas dataframe to process.
    col (str): The column name with the hour values (0 to 23).

    Returns:
    DataFrame: The dataframe with two additional columns 'Hour_Sin' and 'Hour_Cos'.
    """
    time_delta_hours_sin = np.sin(2 * np.pi * df[col] / 24)
    time_delta_hours_cos = np.cos(2 * np.pi * df[col] / 24)
    df['Hour_Sin'] = time_delta_hours_sin
    df['Hour_Cos'] = time_delta_hours_cos

    return df

def sin_cos_24hr_decoder(df, sin_t, cos_t):
    """
    This function takes a dataframe and the column names for sine and cosine
    values that represent time. It calculates the corresponding hour of the day
    for each row and creates a new column 'hour_int' with these values.
    
    Parameters:
    df (DataFrame): The pandas dataframe to process.
    sin_t (str): The column name for the sine of time values.
    cos_t (str): The column name for the cosine of time values.
    
    Returns:
    DataFrame: The dataframe with an additional 'hour_int' column.
    """
    # Compute the angle in radians from the sin and cos values
    angle_rad = np.arctan2(df[sin_t], df[cos_t])
    
   # Ensure the angle is positive for all values
    angle_rad[angle_rad < 0] += 2 * np.pi
    
    # Convert the angle in radians to hours and add as a new column
    df['hour_int'] = round(angle_rad / (2 * np.pi) * 24, 3)
    
    return df

def construct_cellbox_input(Perturbations, Molecular, Phenotype, Output_path):
    """
    Load Perturbation, Molecular, and Phenotype data 
    Where the samples correspond to column names and rows correspond to nodes in the Cellbox output
    
    Parameters:
        Perturbation: Changing experimental conditions 
        Molecular: Measured cellular response 
        Phenotype: Measured output corresponding to phenotype of interest
        Output_Directory: Where the results should be written
        
    Returns:
        expression_matrix: a matrix of values corresponding to the indivual input values compiled for cellboxs
        perturbation_matrix: a matrix of values corresponding to initial base condition values (0) and associated perturbations
        node_index: the names of all individual nodes (rownames)
        
    
    """
    
    experiment = Output_path.split("/")[-2]
    n_protein_nodes = Molecular.shape[0]
    n_activity_nodes = n_protein_nodes + Phenotype.shape[0]
    n_x = n_activity_nodes + Perturbations.shape[0]

    config_json = construct_cellbox_json(experiment, n_protein_nodes, n_activity_nodes, n_x)

    activity_nodes = pd.merge(Molecular.T, Phenotype.T, left_index=True, right_index=True)
    expr_matrix = pd.merge(activity_nodes, Perturbations.T, left_index=True, right_index=True)

    
    #expr_matrix = pd.concat([Molecular, Phenotype, Perturbations]).T
    # Construct a dataframe where all initial values are zero in the shape of (# perturbations + # phenotypes, # samples)
    # Add corrresponding perturbation data
    pert_matrix = np.concatenate([np.zeros((Molecular.shape[0]+Phenotype.shape[0], Molecular.shape[1])),
                                  Perturbations]).T
    
    node_index = expr_matrix.columns.to_series()
    sample_order = expr_matrix.index.to_series()
    
    os.makedirs(Output_path, exist_ok=True)

    with open(os.path.join(Output_path, 'config.json'), 'w') as json_file:
        json.dump(config_json, json_file)

    np.savetxt(os.path.join(Output_path, 'expr_matr.csv'), expr_matrix.to_numpy(), delimiter=",")
    np.savetxt(os.path.join(Output_path, 'pert_matr.csv'), pert_matrix, delimiter=",")
    node_index.to_csv(os.path.join(Output_path, 'node_index.csv'), header=False, index=False)
    sample_order.to_csv(os.path.join(Output_path, 'sample_order.csv'), header=False, index=False)

def construct_cellbox_json(experiment, n_protein_nodes, n_activity_nodes, n_x):
    """
    Constructs a JSON object for the Cellbox experiment with the given parameters.

    Parameters:
    path (str): The base path for the files.
    n_protein_nodes (int): The number of protein nodes.
    n_activity_nodes (int): The number of activity nodes.
    n_x (int): The number of x nodes.

    Returns:
    dict: The constructed JSON object.
    """
    return {
        "experiment_id": experiment,
        "experiment_type": "random partition",
        "model": "CellBox",
        "sparse_data": False,
        "pert_file": f"Execution_files/{experiment}/pert_matr.csv",
        "expr_file": f"Execution_files/{experiment}/expr_matr.csv",
        "node_index_file": f"Execution_files/{experiment}/node_index.csv",
        "n_protein_nodes": n_protein_nodes,
        "n_activity_nodes": n_activity_nodes,
        "n_x": n_x,
        "trainset_ratio": 0.7,
        "validset_ratio": 0.8,
        "batchsize": 32,
        "add_noise_level": 0,
        "envelop_form": "tanh",
        "dT": 0.1,
        "envelop": 0,
        "ode_degree": 1,
        "ode_solver": "heun",
        "ode_last_steps": 2,
        "l1lambda": 1e-4,
        "l2lambda": 1e-4,
        "n_epoch": 10000,
        "n_iter": 10000,
        "n_iter_buffer": 50,
        "n_iter_patience": 100,
        "stages": [
            {
                "nT": 100,
                "sub_stages": [
                    {"lr_val": 0.1, "l1lambda": 0.01, "n_iter_patience": 1000},
                    {"lr_val": 0.01, "l1lambda": 0.01},
                    {"lr_val": 0.01, "l1lambda": 0.0001},
                    {"lr_val": 0.001, "l1lambda": 0.00001}
                ]
            },
            {
                "nT": 200,
                "sub_stages": [
                    {"lr_val": 0.001, "l1lambda": 0.0001}
                ]
            },
            {
                "nT": 400,
                "sub_stages": [
                    {"lr_val": 0.001, "l1lambda": 0.0001}
                ]
            }
        ],
        "export_verbose": 3,
        "ckpt_name": experiment
    }



In [ ]:
A = pd.read_csv("../Data/ica/A.csv", index_col=0)
log2FC  = pd.read_csv("../Data/deseq2/Log2FC_All_Expression.csv", index_col=0)
logTPM_norm = pd.read_csv("../Data/ica/logTPM_counts_normalized_passingQC.csv", index_col=0)
tf_order=pd.read_csv('../Data/deseq2/tf_order.csv',header=None)
# From Puszynska & O'Shea 2017 (DOI: https://doi.org/10.7554/eLife.23210)
light_pert = pd.read_csv("../Data/literature_metadata/circadian_time_light_Puszynska_2017.csv")
light_pert = sin_cos_24hr_encoder(light_pert, 'Circadian_time_h')

## Load the genome annotation
ann = pd.read_csv("../Data/annotation/Selongatus_PCC7942_ref_tfs.csv")
ann.set_index('locus_tag', inplace=True)

# Load all predicted TFs
pred_TFs = pd.read_csv("../Data/annotation/All_predicted_TFs.csv")

# Load all known regulatory interactions
regs = pd.read_csv("../Data/annotation/TRN_complete.csv", index_col=0)

metadata = pd.read_csv('../Data/metadata/Metadata_Perturbation_Passing_QC.csv', index_col=0)
metadata.rename(columns={"Project_tag":"project", "Condition":"condition"}, inplace=True)
metadata['Ref_condition'] = metadata['project'] + "_" + metadata['Ref_condition']
metadata.head()

In [ ]:
markson_ann = pd.read_excel("../Data/literature_metadata/CircadianGenes_Markson_2013.xlsm", skiprows=1)
circadian_genes = markson_ann[markson_ann['Reproducibly circadian'] == 1]
print(f'Number of circadian genes: {circadian_genes.shape[0]}')
#circadian_genes = circadian_genes[~(circadian_genes['Chromosome or plasmid'] == 'pANS')]
circadian_genes.head(10)

## Manual mapping of circadian genes on PANL to alternative annotation using uniprot & Biocyc
## Single pANS gene not included
map_dict = {'HKK26_RS00110': 'Synpcc7942_B2626',
            'HKK26_RS00115': 'Synpcc7942_B2637',
            'HKK26_RS00120': 'Synpcc7942_B2648',
            'HKK26_RS00190': 'Synpcc7942_B2623',
            'HKK26_RS00235': 'Synpcc7942_B2655', 
            'HKK26_RS00240': 'Synpcc7942_B2632',
            'HKK26_RS00020': 'Synpcc7942_B2635'}

circadian_genes_up = ann[(ann.old_locus_tag.isin(circadian_genes['Gene ID'])) | (ann.index.isin(map_dict.keys()))]
print(f"Number of genes mapped to annotation: {circadian_genes_up.shape[0]}")
circadian_genes_up.head()

In [ ]:
#Load difference in gene expression data for TFs (1. Input into ICA)
light_pert_samples = light_pert[['SRA', 'Light', 'Hour_Sin', 'Hour_Cos']].set_index('SRA').T
light_pert_samples

oshea_samples_logTPM_norm_TFs = logTPM_norm.loc[pred_TFs.TFs, light_pert_samples.columns]
oshea_samples_logTPM_norm_TFs.index = oshea_samples_logTPM_norm_TFs.index.map('TF_{}'.format)
oshea_samples_logTPM_norm_TFs

light_pert_conditions = light_pert[['Condition_pi', 'Light', 'Hour_Sin', 'Hour_Cos']
                                  ].drop_duplicates().set_index('Condition_pi').T.drop(columns='wt_clearday_dawn_0.5h')

#Load log2FC data (2. Output from DESeq2)
oshea_samples_log2FC_TFs = log2FC.loc[pred_TFs.TFs, light_pert_conditions.columns]
oshea_samples_log2FC_TFs.index = oshea_samples_log2FC_TFs.index.map('TF_{}'.format)

# Modulon response by sample
oshea_IC_sample = A.loc[:, light_pert_samples.columns]

## Average the modulon resposne across conditions 
## wt_clearday_dawn_0.5h removed as it was used as the base condition for all differential expression comparisons

avg_A = []
for sample, group in light_pert.groupby('Condition_pi'):
    avg = oshea_IC_sample[group.SRA].mean(axis=1)
    avg_A.append(avg)
    
avg_A_df = pd.concat(avg_A, axis=1)
avg_A_df.columns = light_pert.groupby('Condition_pi').groups.keys()
avg_A_df = avg_A_df.drop('wt_clearday_dawn_0.5h', axis=1)
avg_A_df = avg_A_df.reindex(light_pert_conditions.columns.values, axis=1)
avg_A_df.head()

# Module model

In [ ]:
circadian_tfs = set(circadian_genes_up.index).intersection(set(pred_TFs.TFs))
add_regs = ['RpaA', 'RpaB', 'rre1', 'HrcA', 'NtcA']
add_regs_locus_tags = regs[regs.Regulator_Name.isin(add_regs)].regulatoryGene.unique()


print(f'Known Regulators identified in circadian phase: {regs[regs.regulatoryGene.isin(circadian_tfs)].Regulator_Name.unique()}')
print(f'Inclusion of known regulators: {regs[regs.regulatoryGene.isin(circadian_tfs.union(set(add_regs_locus_tags)))].Regulator_Name.unique()}')

circ_log2FC_TFs = oshea_samples_log2FC_TFs.loc[list(["TF_" + TF for TF in circadian_tfs.union(set(add_regs_locus_tags))]),:]
circ_log2FC_TFs = circ_log2FC_TFs.reindex(tf_order[0].values)
construct_cellbox_input(light_pert_conditions, circ_log2FC_TFs, avg_A_df, '../Execution_files/Cellbox_Module')


# Gene model

In [ ]:
circ_log2FC_genes = log2FC[log2FC.index.isin(circadian_genes_up.index)]

construct_cellbox_input(light_pert_conditions, circ_log2FC_TFs, circ_log2FC_genes, 
                        "../Execution_files/Cellbox_Gene")
